In [1]:
## import sklearn
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
from sklearn.linear_model import SGDClassifier
from konlpy.tag import Mecab
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

mecab = Mecab()
#fm.findSystemFonts()
plt.rcParams['font.family']= ["NanumGothicCoding"]
plt.rcParams["axes.unicode_minus"]=False
# GPU 환경 설정하기
# assert tf.test.is_gpu_available() == True, 'GPU 설정을 확인하세요.'
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_logical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


In [2]:
# 데이터를 가져옵니다.
spam = pd.read_csv('./AI_MODEL_2022-10-12_09-48-48/spam.csv')

In [3]:
spam.head()

,text,label
0,[Web발신]박춘규회원님손절주식은그만월급배만드는법http://lco.jp/eA,spam
1,[Web발신]골든브릿지에서손실을보셨나요??http://bitly.kr/bRGtq[FW],spam
2,[Web발신][대//박]가(.원)입코(WN)드MEP.com,spam
3,[Web발신](광고)이정미님아직도주식하시나요http://pf.kakao.com/_u...,spam
4,[Web발신]송승용님사람이모이는곳에는이유가있습니다하루~정보공개is.gd/JsJP,spam


In [4]:
# label 데이터를 수치형으로 변환합니다.
spam['label'] = spam['label'].map({'spam':1, 'ham':0})

In [5]:
spam.dropna(inplace=True)

In [6]:
import nltk
from konlpy.tag import Mecab

mecab = Mecab()
spam['nouns'] = [mecab.nouns(i) for i in spam['text']]
spam['morphs'] = [mecab.morphs(i) for i in spam['text']]
spam

,text,label,nouns,morphs
0,[Web발신]박춘규회원님손절주식은그만월급배만드는법http://lco.jp/eA,1,"[발신, 박춘규, 회원, 손절, 주식, 월급, 배, 법]","[[, Web, 발신, ], 박춘규, 회원, 님, 손절, 주식, 은, 그만, 월급,..."
1,[Web발신]골든브릿지에서손실을보셨나요??http://bitly.kr/bRGtq[FW],1,"[발신, 골든, 브릿지, 손실]","[[, Web, 발신, ], 골든, 브릿지, 에서, 손실, 을, 보, 셨, 나요, ..."
2,[Web발신][대//박]가(.원)입코(WN)드MEP.com,1,"[발신, 대, 박, 원, 코]","[[, Web, 발신, ], [, 대, /, /, 박, ], 가, (, ., 원, ..."
3,[Web발신](광고)이정미님아직도주식하시나요http://pf.kakao.com/_u...,1,"[발신, 광고, 이정미, 주식]","[[, Web, 발신, ], (, 광고, ), 이정미, 님, 아직, 도, 주식, 하..."
4,[Web발신]송승용님사람이모이는곳에는이유가있습니다하루~정보공개is.gd/JsJP,1,"[발신, 송승용, 사람, 곳, 이유, 하루, 정보, 공개]","[[, Web, 발신, ], 송승용, 님, 사람, 이, 모이, 는, 곳, 에, 는,..."
...,...,...,...,...
20095,"[Web발신]이상투자황기봉님""이재훈""투자컨설담당자배정완료.담당자확인▼http://b...",1,"[발신, 상투, 자황, 기봉, 이재훈, 투자, 컨, 설, 담당자, 배정, 완료, 담...","[[, Web, 발신, ], 이, 상투, 자황, 기봉, 님, "", 이재훈, "", 투..."
20096,[Web발신](광고)조마?峙???충?矛?%:까지 ↓주소Joma.카JM톡무료거부,1,"[발신, 광고, 조마, 충, 矛, 주소, 카, 톡, 무료, 거부]","[[, Web, 발신, ], (, 광고, ), 조마, ?, 峙, ?, ??, 충, ..."
20097,[Web발신]김환택님지금이라도늦지않았습니다.월긴급정보http://pf.kakao.c...,1,"[발신, 김, 환택, 지금, 월, 긴급, 정보]","[[, Web, 발신, ], 김, 환택, 님, 지금, 이라도, 늦, 지, 않, 았,..."
20098,[Web발신]년노하우명품구매대행논스토리nonstory.com항상아낌없는사랑과성원에감...,0,"[발신, 년, 노하우, 명품, 구매, 대행, 스토리, 사랑, 성원, 감사]","[[, Web, 발신, ], 년, 노하우, 명품, 구매, 대행, 논, 스토리, no..."


In [7]:
def nouns_sum(nouns):
    return ' '.join(nouns)
spam['text'] = spam['nouns'].apply(nouns_sum)

In [8]:
# train validation set으로 분리합니다.
x = spam['text']
y = spam['label']

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=2022)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((16071,), (4018,), (16071,), (4018,))

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

NGRAM_RANGE = (1, 2)

# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Whether text should be split into word or character n-grams.
# One of 'word', 'char'.
TOKEN_MODE = 'word'

# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2

def ngram_vectorize(train_texts, train_labels, val_texts):
    """Vectorizes texts as n-gram vectors.

    1 text = 1 tf-idf vector the length of vocabulary of unigrams + bigrams.

    # Arguments
        train_texts: list, training text strings.
        train_labels: np.ndarray, training labels.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val: vectorized training and validation texts
    """
    # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
            'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': TOKEN_MODE,  # Split text into word tokens.
            'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn vocabulary from training texts and vectorize training texts.
    x_train = vectorizer.fit_transform(train_texts)

    # Vectorize validation texts.
    x_val = vectorizer.transform(val_texts)
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    
    return x_train, x_val

x_train_ngram, x_val_ngram = ngram_vectorize(x_train, y_train, x_val)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:1804: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


In [10]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

# Vectorization parameters
# Limit on the number of features. We use the top 20K features.
TOP_K = 30000

# Limit on the length of text sequences. Sequences longer than this
# will be truncated.
MAX_SEQUENCE_LENGTH = 500

def sequence_vectorize(train_texts, val_texts):
    """Vectorizes texts as sequence vectors.

    1 text = 1 sequence vector with fixed length.

    # Arguments
        train_texts: list, training text strings.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val, word_index: vectorized training and validation
            texts and word index dictionary.
    """
    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    # Vectorize training and validation texts.
    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH
    print(max_length)
    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val, tokenizer.word_index

x_train_seq, x_val_seq, word_idx = sequence_vectorize(x_train, x_val)

40


In [11]:
# N-grams 방식으로 vectorize한 데이터의 shape을 확인해봅니다.
# 모델 학습시에 활용 가능하도록 전처리 데이터를 저장해보도록 하겠습니다.
import pickle
x_train_ngram


with open('./x_train_ngram', 'wb') as f:
    pickle.dump(x_train_ngram, f)
with open('./x_val_ngram', 'wb') as f:
    pickle.dump(x_val_ngram, f)

In [12]:
# Sequence 형태로 vectorize한 데이터의 shape을 확인해봅니다.
# 모델 학습시에 활용 가능하도록 전처리 데이터를 저장해보도록 하겠습니다.
with open('./x_train_seq', 'wb') as f:
    pickle.dump(x_train_seq, f)
with open('./x_val_seq', 'wb') as f:
    pickle.dump(x_val_seq, f)

In [13]:
# label 데이터의 shape을 확인하고 저장합니다.
with open('./y_train', 'wb') as f:
    pickle.dump(y_train, f)
with open('./y_val', 'wb') as f:
    pickle.dump(y_val, f)